### 1. 해킹 - B10282
> 다익스트라
- 오랜만에 보니 너무 반갑다 ^^ heapq 를 import해야함
- 그냥 다익스트라 풀고 INF아닌것 카운트 하면 되는 문제

> 다익스트라의 시간복잡도는 ElogE, 간선의 개수가 절대적인 영향
- O(ElogE)

In [2]:
from heapq import heappush,heappop

INF = 1e9

def dijkstra(graph,start):
    distance = [INF]*(N+1)
    distance[start] = 0
    queue = []
    heappush(queue,[0,start])
    while queue:
        cost,pos = heappop(queue)
        for p, c in graph[pos]:
            c += cost
            if distance[p] > c:
                distance[p]=c
                heappush(queue,[c,p])
    return distance

for _ in range(int(input())):
    # D 신뢰 개수 , C = start_node라 생각하자
    N,D,C = map(int,input().split())
    Graph = [[]for _ in range(N+1) ]
    for _ in range(D):
        # B 에서 S초 후 A로 감(일방향)
        A,B,S = map(int,input().split())
        Graph[B].append((A,S))
    ANS = dijkstra(Graph,C)
    result=[]
    final=[]
    count=0
    for i in ANS:
        if i !=INF:
            count+=1
            result.append(i)
    final.append([count,max(result)])
    for i in range(len(final)):
        for j in range(len(final[i])):
            print(final[i][j], end=' ')
        print()

1
3 2 2
2 1 5
3 2 5
5


In [4]:
final = [[2,5],[3,6]]
for i in range(len(final)):
    for j in range(len(final[i])):
        print(final[i][j], end=' ')
    print()

2 5 
3 6 


### 2. 거의 최단경로 - B5719
> 다익스트라 & BFS
- 개빡치는 문제다.
- 다익스트라 수행후, 최단경로에 포함되는 모든 간선을 추적! 
- 초기 최단경로 포함 간선 제외후, 다시 최단경로 탐색 (다익스트라X2)

> 풀어본 후 핵심 point
- 처음에 생성하는 연결 그래프를, 역방향그래프도 동시에 만들어야 추적에 용이
- 다익스트라 함수밖에서 distance를 생성.
- 조건에 not dropped[pos][p] 를 설정. 초기에는 그냥 지나갈 것
- BFS() 에서 역으로 추적할때 distance를 사용
- BFS()에서 연결되는 간선들은 dropped[출발][다음간선] 을 True로 
- distance 초기화 후 다익스트라 재 탐색, dropped가 첫번째 최단 제외하고 탐색


#### 추적! 이란?
> 최단경로상 도착노드부터, 인접노드의 가중치를 파악해 추적함
- 이를 위해 BFS를 활용함

In [1]:
from collections import deque
from heapq import heappush,heappop
INF = 1e9

def dijkstra(Graph,start):
    distance[start] = 0
    queue = []
    heappush(queue,[0,start])
    while queue:
        cost, pos = heappop(queue)
        for p,c in Graph[pos]:
            c += cost
            if distance[p] > c and not dropped[pos][p]:
                distance[p] = c
                heappush(queue,[c,p])
                
def BFS(Graph,start_rev):
    queue = deque([end])
    visited = [False] *(N+1)
    visited[start_rev] = True
    while queue:
        node = queue.popleft()
        if node == start:
            continue
        for prev, cost in Graph[node]:
            if distance[node] == distance[prev] + cost:
                dropped[prev][node] = True
                queue.append(prev)

In [2]:
while True:
    N,M = map(int,input().split())
    if N ==0:
        break
    start,end = map(int,input().split())
    adj = [[] for _ in range(N+1)]
    reverse_adj = [[] for _ in range(N+1)]
    for _ in range(M):
        x,y,cost = map(int,input().split())
        adj[x].append((y,cost))
        reverse_adj[y].append((x,cost))
    dropped = [[False]*(N+1) for _ in range(M+1)]
    distance = [INF] * (N+1)
    dijkstra(adj,start)
    BFS(reverse_adj,end)
    distance = [INF] * (N+1)
    dijkstra(adj,start)
    if distance[end] != INF:
        print(distance[end])
    else:
        print(-1)

1 1
1111


ValueError: not enough values to unpack (expected 2, got 1)

### 3. 우주신과의 교감 - B1774

> 주어진 2차원 좌표를 이용해, 모든 좌표를 잇는 MST를 만들어야 함
- 2차원 좌표상의 점을 잇는 통로들을 고려해야함
- 정점의 개수 N이 최대 1000, 가능한 통로의 개수는 약 N^2
- Kruskal은 간선이 E일때 O(ElogE)로 동작

##### 따라서 크루스칼 알고리즘으로 해결 할 수 있음

#### 기존의 크루스칼 알고리즘 - Dave way

In [9]:
# graph는 vertices:[A,B,C....], edges:[(7,A,B), .... ]
# 이런식의 dict()자료형
parent = dict()
rank = dict()

def find(node):
    if parent[node] != node:
        parent[node] =  find(parent[node])
    return parent[node]

def union(node_v,node_u):
    root1 = find(node_v)
    root2 = find(node_u)
    
    if rank[root1] > rank[root2]:
        parent[root2] = root1
    else:
        parent[root1] = root2
        if rank[root1] == rank[root2]:
            rank[root2]+=1
            
def make_set(node):
    parent[node] = node
    rank[node] = 0

def kruskal(graph):
    mst = list()
    
    for node in graph['vertices']:
        make_set(node)
    edges = graph['edges']
    edges.sort()
    
    for edge in edges:
        weight, node_v,node_u = edge
        if find(node_v) != find(node_u):
            union(node_v,node_u)
            mst.append(edge)
    return mst

#### 우주신과의 교감- [해답]

In [3]:
import math

parent = {}

def get_distance(p1,p2):
    a = p1[0] - p2[0]
    b = p1[1] - p2[1]
    return math.sqrt((a*a)+(b*b))

def get_parent(parent,n):
    if parent[n] == n:
        return n
    return get_parent(parent,parent[n])

def union(parent,a,b):
    a = get_parent(parent,a)
    b = get_parent(parent,b)
    if a<b:
        parent[b] = a
    else:
        parent[a] = b

def find(parent,a,b):
    a = get_parent(parent,a)
    b = get_parent(parent,b)
    if a== b:
        return True
    else:
        return False

In [8]:
N,M = map(int,input().split())
locations = []
edges = []

# N개의 우주신의 위치 (X,Y) 를 location리스트에 모두 저장

for _ in range(N):
    X,Y = map(int,input().split())
    locations.append((X,Y))
    
length = len(locations)

# 모든 우주신들 사이의 거리를 edges라는 리스트에 추가
for i in range(length-1):
    for j in range(i+1,length):
        edges.append((i+1,j+1,get_distance(locations[i],locations[j])))

# 모든 우주신들의 parent는 자신으로 초기화
for i in range(1,N+1):
    parent[i] = i
    
#  M개의 존재하는 통로를 설정, 거리길이가 오름차순이 되도록 정렬
for i in range(M):
    a,b = map(int,input().split())
    union(parent,a,b)
edges.sort(key = lambda data:data[2])

result = 0
# 짧은 거리부터 우주신 a,b의 root가 일치하는지 확인후 union
# 앞서 설정한 존재하는 통로는 union되어 있는 상태
# root가 다른경우(추가되는 경우)만 cost를 합산, 최종 통로길이
for a, b, cost in edges:
    if not find(parent,a,b):
        union(parent,a,b)
        result += cost
        
print("%.2f" %result)

4 1
1 1
3 1
2 3
4 3
1 4
4.00
